# Part 4: Using GPU acceleration with PyTorch

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 2.9MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [0]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 0.3638,  0.2822, -0.0519,  ..., -0.9773,  0.1531, -0.7892],
        [ 0.5570, -1.4245,  0.1477,  ...,  0.1743,  1.3716,  0.8632],
        [-0.8996,  0.4613,  0.7738,  ..., -0.1370,  0.0114, -0.7012],
        ...,
        [-0.2455,  2.3780, -2.5666,  ...,  0.1597, -0.1841, -0.0242],
        [ 1.0648,  0.5747, -0.1149,  ...,  0.1512, -0.1807, -1.0339],
        [ 0.0139, -0.1626, -0.0213,  ...,  0.0449,  2.8371, -0.7697]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [0]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[ 1.1197,  1.5118,  0.7204,  ...,  0.0828, -0.0334, -3.1598],
        [-3.9197,  0.1426,  0.1883,  ...,  0.2123, -1.5003, -0.0417],
        [ 0.6493, -1.6372, -0.8026,  ...,  1.0369, -0.6957, -0.1569],
        ...,
        [ 0.1198,  0.0270,  0.0217,  ..., -0.4998,  0.7630,  3.0304],
        [ 0.5643,  0.5878, -0.0373,  ..., -0.0622,  0.4614,  0.1343],
        [-0.4306,  0.2385,  0.2142,  ...,  0.1161,  0.1544,  1.8255]],
       device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [0]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-1.6558e-01,  4.7876e-01, -7.8647e-01,  ...,  4.8021e-04,
          2.2277e-02,  5.1901e-01],
        [ 8.2329e-03,  2.5940e-01,  1.1447e+00,  ...,  8.3060e-01,
         -5.5657e-01, -3.6138e-01],
        [-6.5229e-01, -8.9905e-02,  9.9884e-02,  ..., -4.5401e-01,
          1.9141e+00, -7.2981e-02],
        ...,
        [ 3.5855e-01, -3.6749e-02, -1.4461e-02,  ...,  2.3154e-01,
         -3.4901e-02,  5.7503e-01],
        [ 1.1992e-01, -3.3649e-02,  3.2435e-01,  ...,  8.0512e-02,
          5.9869e-02,  5.9768e-01],
        [-6.1775e-01,  1.4112e-01,  1.5497e-01,  ...,  1.0519e+00,
          2.1384e-01,  7.8189e-01]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

Single Instruction Multiple Data (SIMD) architecture, that dsitributes a single instruction across mutiple data blocks.

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

Lack of memory (RAM) to store weights and activation functions 